In [14]:
import numpy as np
import scipy.constants as spc
import pandas as pd
import altair as alt

In [3]:
# Constants
mu_0 = spc.mu_0
pi = spc.pi
mag_const = mu_0/(4*pi)

In [20]:
D=pd.read_table("DATALOG.TXT",names=['date','time','position','voltage'])
pos=pd.DataFrame({'point':['begin','end'],'latitude':[47.250755, 47.250771],'longitude':[-122.480139,-122.480191]})
print(pos)
D['latitude']=np.linspace(pos['latitude'][0],pos['latitude'][1],len(D['position']))
D['magnetic_field']=D['voltage'].str.replace(' uV','').astype(float)/(4*12.2*100)
print(D)

   point   latitude   longitude
0  begin  47.250755 -122.480139
1    end  47.250771 -122.480191
          date          time  position     voltage   latitude  magnetic_field
0    2000-0-26  13:29:30.000       NaN   -77832 uV  47.250755      -15.949180
1    2000-0-26  13:29:32.000       NaN   -77456 uV  47.250755      -15.872131
2    2000-0-26  13:29:34.000       NaN   -77268 uV  47.250755      -15.833607
3    2000-0-26  13:29:36.250       NaN   -77268 uV  47.250755      -15.833607
4    2000-0-26  13:29:38.250       NaN   -77268 uV  47.250756      -15.833607
..         ...           ...       ...         ...        ...             ...
115  2021-7-15   16:8:41.099       NaN   225600 uV  47.250770       46.229508
116  2021-7-15   16:8:43.128       NaN   227856 uV  47.250771       46.691803
117  2021-7-15   16:8:45.158       NaN   227668 uV  47.250771       46.653279
118  2021-7-15   16:8:47.187       NaN   211124 uV  47.250771       43.263115
119  2021-7-15   16:8:49.217       NaN  -16544

In [21]:
# Plot anomaly field
measured_plot=alt.Chart(D).mark_point(color='steelblue',shape='square').encode(
    x=alt.X("latitude:Q",scale=alt.Scale(domain=(-100,100))),
    y=alt.Y("magnetic_field:Q")
)
display(measured_plot)

alt.Chart(...)

In [ ]:
# Set up survey parameters
D = 100   # Line length in m
h = 2      # Station height in m above ground
N=1000     # Number of points

survey_line=pd.DataFrame({'x':[np.array(xi) for xi in np.vstack([np.linspace(-D,D,N),np.ones((N,))*h]).T]})

In [ ]:
# Set up Earth's magnetic field
latitude = 47 # Latitude
I_Earth = np.arctan(2*np.tan(np.radians(latitude)))  # Inclination of Earth's field
B_Earth = 50E-6  # Earth's field magnitude in T (Note: B field, need to divide by mu_0 to get H)

In [ ]:
# Set up source parameters

yd = 1.     # Depth of dipole in m
I_dipole = np.radians(80) # Inclination of dipole in degrees (assume N pole is on +X side)
    # Set equal to I_Earth for induced magnetization
mu_magnitude = 1. # Dipole moment magnitude in Am^2

# Calculate dipole moment vector
mu = mu_magnitude*np.array([np.cos(I_dipole), -np.sin(I_dipole)])

def dipole(x_survey, x_dipole, mu):
    r = x_survey-x_dipole
    r_magnitude = np.linalg.norm(r)
    r_hat = r/r_magnitude
    return mag_const*3*((np.dot(mu,r_hat)*r_hat)-mu)/(r_magnitude**3)
    

survey_line['B'] = survey_line['x'].apply(dipole,x_dipole=np.array([0, -yd]), mu=mu)
print(survey_line)

In [ ]:
# Calculate anomaly field
survey_line['delta_B']=survey_line['B'].apply(lambda xi: np.dot(xi,np.array([B_Earth*np.cos(np.radians(I_Earth)), 
                                                                             -B_Earth*np.sin(np.radians(I_Earth))])))

In [ ]:
# Break vectors x and B into components for plotting
survey_line['transect_distance']=survey_line['x'].apply(lambda xi: xi[0])
survey_line['Bh']=survey_line['B'].apply(lambda xi: xi[0])
survey_line['Bv']=survey_line['B'].apply(lambda xi: xi[1])

In [ ]:
# Plot anomaly field
anomaly_plot=alt.Chart(survey_line).mark_line().encode(
    x=alt.X("transect_distance:Q",scale=alt.Scale(domain=(-100,100))),
    y=alt.Y("delta_B:Q")
)
display(anomaly_plot)